## 进程线程


一. fork()创建子进程特点
    1. 子进程会复制父进程全部的内存空间和代码段
    2. 子进程会从fork的下一句开始执行
    3. if...elif...else结构正是根据父子进程fork的返回值不同让父子进程执行不同的内容，几乎是fork的固    定搭配
    4. 父子进程运行互不影响，使用同一个终端，运行顺序不定
    5. 子进程有自己特有的PID，PCB，命令集等
    6. 进入阻塞状态的进程一定会让出cpu时间片
    7. 子进程连同fork之前开辟的空间也会复制。但是父子进程各自空间独立，操作各自空间内容，互不影响

二、进程相关函数使用
    
    os.getpid()
    功能： 获取当前进程的PID号
    返回值： 返回PID号

    os.getppid()
    功能: 获取父进程的PID号
    返回值：返回 父进程PID号

    os._exit(status)
    功能： 退出一个进程
    参数： 表示进程退出状态 

    sys.exit([status])
    功能： 退出一个进程
    参数： 默认为0 整数表示进程退出状态 
           字符串 则在进程退出时打印该字符串

二、进程相关函数使用
    
    os.getpid()
    功能： 获取当前进程的PID号
    返回值： 返回PID号

    os.getppid()
    功能: 获取父进程的PID号
    返回值：返回 父进程PID号

    os._exit(status)
    功能： 退出一个进程
    参数： 表示进程退出状态 

    sys.exit([status])
    功能： 退出一个进程
    参数： 默认为0 整数表示进程退出状态 
           字符串 则在进程退出时打印该字符串


In [ ]:
# 最简单的进程创建方法
import os
from time import sleep 

pid = os.fork()

if pid < 0:
    print("Create process error")
elif pid == 0:
    sleep(2)
    print("New process")
else:
    sleep(3)
    print("The old process")

print("fork test end...")

In [ ]:
# 主进程等待子进程结束后退出
import os 
from time import sleep 

pid = os.fork()

if pid < 0:
    print("Create process failed")
elif pid == 0:
    sleep(3)
    print("Child %d process exit"%os.getpid())
    os._exit(3)
else:
    #阻塞等待处理子进程退出
    pid,status = os.wait()
    print("pid:",pid) #退出的子进程PID
    print("status:",os.WEXITSTATUS(status)) #子进程退出状态
    print("Parent process...")
    while True:
        pass 

## Python创建进程

In [2]:
from multiprocessing import Process 
from time import sleep 

#带参数的进程函数
def worker(sec,name):
    for i in range(3):
        sleep(sec)
        print("I'm %s"%name)
        print("I'm working...")

# 按照位置传递参数
# p = Process(target=worker,args=(2,'Levi'))

# 按照键的名称传递参数
p = Process(target = worker,args = (2,),\
kwargs = {'name':"Levi"})

p.start()
p.join()

I'm Levi
I'm working...
I'm Levi
I'm working...
I'm Levi
I'm working...


In [3]:
# 自定义进程类
from multiprocessing import Process 
import time 

class ClockProcess(Process):
    def __init__(self,value):
        self.value = value
        super().__init__()

    # def funcname(self, parameter_list):
    #     pass

    #重写run方法
    def run(self):
        for i in range(5):
            print("The time is",time.ctime())
            time.sleep(self.value)

#创建自定义类进程对象
p = ClockProcess(2)
p.start()  #自动调用run
p.join()

The time is Sun Jul 21 20:11:46 2019
The time is Sun Jul 21 20:11:48 2019
The time is Sun Jul 21 20:11:50 2019
The time is Sun Jul 21 20:11:52 2019
The time is Sun Jul 21 20:11:54 2019


## 进程池

In [5]:
from multiprocessing import Pool 
from time import sleep,ctime 

#事件函数
def worker(msg):
    sleep(2)
    print(msg)
    return ctime()

#创建进程池
pool = Pool(4)

result = []

#向进程池添加事件
for i in range(10):
    msg = "hello %d"%i

    # 异步执行: 多个一同执行
    r = pool.apply_async(func=worker,args=(msg,))
    result.append(r)

    # 同步执行: 一个一个执行
    #pool.apply(func=worker,args=(msg,))

#关闭进程池
pool.close()
#回收进程
pool.join()

for i in result:
    print(i.get()) #可以获取进程事件函数返回值



hello 0
hello 3
hello 1
hello 2
hello 4
hello 5
hello 6
hello 7
hello 8
hello 9
Sun Jul 21 20:15:23 2019
Sun Jul 21 20:15:23 2019
Sun Jul 21 20:15:23 2019
Sun Jul 21 20:15:23 2019
Sun Jul 21 20:15:25 2019
Sun Jul 21 20:15:25 2019
Sun Jul 21 20:15:25 2019
Sun Jul 21 20:15:25 2019
Sun Jul 21 20:15:27 2019
Sun Jul 21 20:15:27 2019


In [4]:
from multiprocessing import Pool 
import time 

def fun(n):
    time.sleep(1)
    return n ** 2

pool = Pool()

#使用map将事件放入进程池
r = pool.map(fun,[1,2,3,4,5])
pool.close()
pool.join()
print(r)

[1, 4, 9, 16, 25]


## 进程间通信

### 管道通信

In [8]:
from multiprocessing import Process,Pipe 
import os,time 

#创建管道对象
# fd1 --> 只读 recv
# fd2 --> 只写 send
fd1,fd2 = Pipe(False)

def fun(name):
    time.sleep(3)
    fd2.send([1,2,3,4,5])

jobs = []
for i in range(5):
    p = Process(target = fun,args = (i,))
    jobs.append(p)
    p.start()
print(jobs)
#父进程从管道读取消息
for i in range(5):
    data = fd1.recv()
    print(data)

for i in jobs:
    i.join()


[<Process(Process-21, started)>, <Process(Process-22, started)>, <Process(Process-23, started)>, <Process(Process-24, started)>, <Process(Process-25, started)>]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]


## 消息列队

In [10]:
from multiprocessing import Process,Queue
import time 

#创建消息队列
q = Queue()

def fun1():
    for i in range(5):
        time.sleep(1)
        q.put((10,2))  #存储消息

def fun2():
    for i in range(3):
        time.sleep(1.5)
        a,b = q.get() #取出消息
        print("sum = ",a + b)

p1 = Process(target = fun1)
p2 = Process(target = fun2)
p1.start()
p2.start()
p1.join()
p2.join()

sum =  12
sum =  12
sum =  12


## 共享内存

In [14]:
from multiprocessing import Process,Array 
import time 




#创建共享内存
# shm = Array('i',[1,2,3,4])

#开辟5个int空间
# shm = Array('i',5)

#存入字符串
shm = Array('c',b'Hello')


def fun():
    for i in shm:
        print(i,'fun')
    shm[0] = b'h' #修改共享内存

p = Process(target = fun)
p.start()
p.join()

for i in shm:
    print(i,'shm')

print(shm.value,'value') #打印字符串


b'H' fun
b'e' fun
b'l' fun
b'l' fun
b'o' fun
b'h' shm
b'e' shm
b'l' shm
b'l' shm
b'o' shm
b'hello' value


## 信号量

In [15]:
from multiprocessing import Process,Semaphore 
from time import sleep 
import os 

#创建信号量
sem = Semaphore(3)

def fun():
    print("%d 想执行事件"%os.getpid())
    #消耗一个信号量
    sem.acquire()
    print("%d 执行事件......"%os.getpid())
    sleep(3)
    print("%d 执行完毕"%os.getpid())
    sem.release() #执行完成后添加一个信号量

jobs = []
#5个进程想执行事件
for i in range(5):
    p = Process(target = fun)
    jobs.append(p)
    p.start()

for i in jobs:
    i.join()

print("Sem:",sem.get_value()) #最后剩3个

26765 想执行事件
26765 执行事件......
26768 想执行事件
26768 执行事件......
26771 想执行事件
26772 想执行事件
26775 想执行事件
26771 执行事件......
26765 执行完毕
26772 执行事件......
26768 执行完毕
26775 执行事件......
26771 执行完毕
26772 执行完毕
26775 执行完毕
Sem: 3


## 线程

In [ ]:
from threading import Thread 
from time import sleep 

#线程函数
def fun(sec,name):
    print("线程函数传参")
    sleep(sec)
    print("%s线程执行完毕"%name)

#创建多个线程
thread = []
for i in range(3):
    t = Thread(target=fun,args=(2,),\
    kwargs={'name':"t%d"%i})
    thread.append(t)
    t.start()

for i in thread:
    i.join()


In [ ]:
from threading import Thread,currentThread 
from time import sleep 

def fun():
    print("当前线程:",currentThread().getName())
    sleep(3)
    print("线程属性示例")

#创建线程对象
t = Thread(target=fun,name="Tedu")

#设置daemon
t.setDaemon(True)
print("Daemon:",t.isDaemon())

t.start()

#线程名称
t.setName("tarena")
# print("name:",t.name)
print("name:",t.getName())

#线程状态
print("Is alive:",t.is_alive())

# t.join()


## 线程事件

In [ ]:
from threading import Thread,Event 
from time import sleep 

s = None  #设置为通信变量
e = Event()

def bar():
    print("Bar 拜山头")
    sleep(1)
    global s 
    s = "天王盖地虎"
    e.set()

b = Thread(target=bar)
b.start()

print("说对口令就是自己人")
e.wait() #阻塞等待,分支线程set
if s == "天王盖地虎":
    print("确认过眼神,你是对的人")
else:
    print("打死他")

b.join()

## 线程锁

In [ ]:
from threading import Thread,Lock 

a = b = 0
lock = Lock()

def value():
    while True:
        lock.acquire() #加锁
        if a != b:
            print("a = %d,b = %d"%(a,b))
        lock.release() #解锁

t = Thread(target = value)
t.start()

while True:
    with lock:   #加锁
        a += 1
        b += 1

t.join()

## 封装线程类

In [ ]:
from threading import Thread 
from time import sleep,ctime 

#自定义线程类
class MyThread(Thread):
    def __init__(self,target,args=(),kwargs={},\
    name='Tedu'):
        super().__init__()
        self.target = target 
        self.args = args
        self.kwargs = kwargs
        self.name = name 
    
    def run(self):
        self.target(*self.args,**self.kwargs)

#测试函数
def player(sec,song):
    for i in range(2):
        print("Playing %s:%s"%(song,ctime()))
        sleep(sec)

t = MyThread(target=player,args=(3,),\
kwargs={'song':'凉凉'},name='Tedu')
t.start()
t.join()

